# Agents with examples

In [ ]:
# %uv pip install agentor

## What is an Agent?

<img src="https://i.imgflip.com/a7rr5a.jpg" width="250px">

<br>

## Agents are LLMs with higher autonomy

- Tool access
- Memory

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from openai import OpenAI

client = OpenAI()
response = client.responses.create(
    model="gpt-5-nano", input="Write a haiku about recursion in programming."
)

print(response.output[-1].content[0].text)

In [ ]:
from agentor import Agentor, tool

agent = Agentor(
    name="Assistant",
    instructions="You are a helpful assistant",
    model="gpt-5-nano",
)

result = await agent.arun("Write a haiku about recursion in programming.")
print(result.final_output)

Recursive echoes
I call myself to solve it
Until base case stops

In [ ]:
agent = Agentor(name="Assistant", instructions="Reply very concisely.")

result = await agent.arun("Write a haiku about recursion in programming.")
print(result.final_output)

Call itself again
until the base case shines clear
Then it calls again

# Tool use

In [ ]:
result = await agent.arun("What is the weather in London?")
print(result.final_output)

I can’t fetch real-time weather here. Want me to look it up (if browsing is enabled) or should I give you links to 
BBC Weather or the Met Office for London?

Use Celesto Tool Hub for a realtime access to weather data and 100+ tools.

```python
from agentor import CelestoSDK

client = CelestoSDK(CELESTO_API_KEY)

# List all available tools
client.toolhub.list_tools()

# Run the weather tool for a specific location
client.toolhub.run_weather_tool("London")
```

In [ ]:
import os

from agentor import CelestoSDK

client = CelestoSDK(os.environ.get("CELESTO_API_KEY"))

# List all available tools
client.toolhub.list_tools()

# Run the weather tool for a specific location
client.toolhub.run_weather_tool("London")

In [ ]:
from agentor import Agentor


@tool
def get_weather(city: str) -> str:
    """returns weather info for the specified city."""
    return f"The weather in {city} is sunny"


agent = Agentor(
    name="Haiku agent",
    instructions="Always respond in haiku form",
    model="gpt-5-nano",
    tools=[get_weather],
)

In [ ]:
result = await agent.arun("What is the weather in London?")

In [ ]:
print(result.final_output)

## LLM with tool use

In [ ]:
weather_tool_def = {
    "type": "function",
    "name": "get_weather",
    "description": "Retrieves current weather for the given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. London, United Kingdom",
            },
            "units": {
                "type": "string",
                "enum": ["celsius", "fahrenheit"],
                "description": "Units the temperature will be returned in.",
            },
        },
        "required": ["location", "units"],
        "additionalProperties": False,
    },
    "strict": True,
}


def get_weather(location: str, units: str) -> str:
    """returns weather info for the specified location and units."""
    return (
        f"The weather in {location} is sunny with a temperature of 20 degrees {units}."
    )


tool_registry = {"get_weather": get_weather}

In [ ]:
client = OpenAI()

response = client.responses.create(
    model="gpt-5-nano", input="What is the weather in London?", tools=[weather_tool_def]
)

In [ ]:
print(response.output[-1])

In [ ]:
import json

if response.output[-1].type == "function_call":
    fn_name = response.output[-1].name
    arguments = json.loads(response.output[-1].arguments)
    result = tool_registry[fn_name](**arguments)
    print(result)
else:
    print(response.output[-1].content[0].text)

## Serve the Agent

```python
from agentor import Agentor

agent = Agentor(
    name="Weather Agent",
    model="gpt-5",
    tools=["get_weather"],
)

# Serve agent with A2A protocol enabled automatically
agent.serve(port=8000)
# Agent card available at: http://localhost:8000/.well-known/agent-card.json
```

--

To put on cloud, run: `celesto deploy`

## MCP - connect private data to Agents


```python
from agentor.mcp import LiteMCP

# Create the app
mcp = LiteMCP()

# Register a tool
@mcp.tool(description="Get weather")
def get_weather(location: str) -> str:
    return f"Weather in {location}: Sunny"

# Run the server
mcp.run()
```

## Security for AI Agent tools

```python
from agentor.tools import SlackTool
from agentor.mcp import LiteMCP, get_headers

mcp = LiteMCP()

@mcp.tool(description="Send a message to a Slack channel")
def send_slack_message(channel: str, message: str):
    headers = get_headers()
    token = headers["authorization"]

    slack = SlackTool(token=token)
    slack.send_message(channel=channel, text=message)

    return "Message sent successfully"

mcp.run()
```